In [1]:
import pandas as pd
from pandas_schema import pandas_schema
from etl_sf_loan_data import get_indices_by_type, preprocess_dates2, preprocess_booleans, convert_to_bool, cast_types

In [1]:
columns = list(pandas_schema.keys())
datatypes = list(pandas_schema.values())
schema_list = list(zip(columns, datatypes))
file_path = '/Users/matthewtryba/Desktop/subsampled_data.csv'
sf_loan_performance = pd.read_csv(file_path, sep='|', header=None, names=columns, low_memory=False)

NameError: name 'pandas_schema' is not defined

In [7]:
sf_loan_performance.head()

,Reference_Pool_ID,Loan_Identifier,Monthly_Reporting_Period,Channel,Seller_Name,Servicer_Name,Master_Servicer,Original_Interest_Rate,Current_Interest_Rate,Original_UPB,UPB_at_Issuance,Current_Actual_UPB,Original_Loan_Term,Origination_Date,First_Payment_Date,Loan_Age,Remaining_Months_to_Legal_Maturity,Remaining_Months_to_Maturity,Maturity_Date,Original_Loan_to_Value_Ratio_LTV,Original_Combined_Loan_to_Value_Ratio_CLTV,Number_of_Borrowers,Debt_To_Income_DTI,Borrower_Credit_Score_at_Origination,Co_Borrower_Credit_Score_at_Origination,First_Time_Home_Buyer_Indicator,Loan_Purpose,Property_Type,Number_of_Units,Occupancy_Status,Property_State,Metropolitan_Statistical_Area_MSA,Zip_Code_Short,Mortgage_Insurance_Percentage,Amortization_Type,Prepayment_Penalty_Indicator,Interest_Only_Loan_Indicator,Interest_Only_First_Principal_And_Interest_Payment_Date,Months_to_Amortization,Current_Loan_Delinquency_Status,...,Borrower_Credit_Score_Current,Co_Borrower_Credit_Score_Current,Mortgage_Insurance_Type,Servicing_Activity_Indicator,Current_Period_Modification_Loss_Amount,Cumulative_Modification_Loss_Amount,Current_Period_Credit_Event_Net_Gain_or_Loss,Cumulative_Credit_Event_Net_Gain_or_Loss,Special_Eligibility_Program,Foreclosure_Principal_Write_off_Amount,Relocation_Mortgage_Indicator,Zero_Balance_Code_Change_Date,Loan_Holdback_Indicator,Loan_Holdback_Effective_Date,Delinquent_Accrued_Interest,Property_Valuation_Method,High_Balance_Loan_Indicator,ARM_Initial_Fixed_Rate_Period_5YR_or_Less_Indicator,ARM_Product_Type,Initial_Fixed_Rate_Period,Interest_Rate_Adjustment_Frequency,Next_Interest_Rate_Adjustment_Date,Next_Payment_Change_Date,Loan_Index,ARM_Cap_Structure,Initial_Interest_Rate_Cap_Up_Percent,Periodic_Interest_Rate_Cap_Up_Percent,Lifetime_Interest_Rate_Cap_Up_Percent,Mortgage_Margin,ARM_Balloon_Indicator,ARM_Plan_Number,Borrower_Assistance_Plan,High_Loan_to_Value_HLTV_Refinance_Option_Indicator,Deal_Name,Repurchase_Make_Whole_Proceeds_Flag,Alternative_Delinquency_Resolution,Alternative_Delinquency_Resolution_Count,Total_Deferral_Amount,Payment_Deferral_Modification_Event_Indicator,Interest_Bearing_UPB
0,NaN,100936387698,2002-07-01 00:00:00,R,Usaa Federal Savings Bank,Other,NaN,8.000,8.000,140000.0,NaN,136301.90,360.0,1999-12-01 00:00:00,2000-02-01 00:00:00,30.0,330.0,329.0,012030,90.0,NaN,2.0,13.0,696.0,648.0,Y,P,SF,1.0,P,TX,19100,752.0,25.0,FRM,N,NaN,NaN,NaN,00,...,NaN,NaN,1.0,N,NaN,NaN,NaN,NaN,7,NaN,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,7,NaN
1,NaN,100416816452,2000-07-01 00:00:00,R,Other,NaN,NaN,8.000,8.000,102000.0,NaN,101866.50,360.0,1999-11-01 00:00:00,2000-01-01 00:00:00,7.0,353.0,353.0,122029,80.0,NaN,1.0,42.0,695.0,NaN,N,P,PU,1.0,P,FL,27260,322.0,NaN,FRM,N,NaN,NaN,NaN,00,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,7,NaN,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,7,NaN
2,NaN,100399640918,2000-06-01 00:00:00,R,Suntrust Mortgage Inc.,NaN,NaN,7.625,7.625,164000.0,NaN,0.00,360.0,1999-12-01 00:00:00,2000-02-01 00:00:00,5.0,355.0,352.0,012030,80.0,NaN,2.0,41.0,787.0,800.0,N,P,SF,1.0,P,GA,12060,301.0,NaN,FRM,N,NaN,NaN,NaN,00,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,7,NaN,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,7,NaN
3,NaN,100394926109,2004-08-01 00:00:00,B,Other,Other,NaN,8.000,8.000,90000.0,NaN,82963.70,360.0,1999-12-01 00:00:00,2000-03-01 00:00:00,54.0,306.0,278.0,022030,72.0,NaN,2.0,50.0,695.0,NaN,N,R,SF,1.0,P,OR,0,978.0,NaN,FRM,N,NaN,NaN,NaN,00,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,7,NaN,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,7,NaN
4,NaN,100650934438,2002-06-01 00:00:00,R,"Jpmorgan Chase Bank, Na","Jpmorgan Chase Bank, Na",NaN,7.625,7.625,205000.0,NaN,196646.27,360.0,1999-12-01 00:00:00,2000-02-01 00:00:00,29.0,331.0,312.0,012030,70.0,NaN,1.0,26.0,735.0,NaN,N,P,SF,1.0,P,MA,14460,24.0,NaN,FRM,N,N

In [6]:
indices_by_type = get_indices_by_type(schema_list)
preprocess_dates2(sf_loan_performance, indices_by_type)

sf_loan_performance.head()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/matthewtryba/anaconda3/envs/tryba-env/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/tf/_d857dk519v0j3_x_bbjf86r0000gn/T/ipykernel_3618/1430024605.py", line 2, in <module>
    preprocess_dates(sf_loan_performance, indices_by_type)
  File "/Users/matthewtryba/Library/CloudStorage/Dropbox/01 Matt/Tryba Documents/GitHub/Portfolio-Projects/05 Mortgage Home Loan Analysis/etl_sf_loan_data.py", line 52, in preprocess_dates
  File "/Users/matthewtryba/anaconda3/envs/tryba-env/lib/python3.10/site-packages/pandas/core/indexing.py", line 849, in __setitem__
    iloc._setitem_with_indexer(indexer, value, self.name)
  File "/Users/matthewtryba/anaconda3/envs/tryba-env/lib/python3.10/site-packages/pandas/core/indexing.py", line 1835, in _setitem_with_indexer
    self._setitem_with_indexer_split_path(indexer, value, name)
  File "/Users/m